# MORPC Insights - Distributed Energy Resources (DER)

## Overview

The Public Utilities Commission of Ohio maintains a [database](https://maps.puco.ohio.gov/arcgis/rest/services/electric/Distributed_Energy_Resources/MapServer/0/) containing the locations and attributes of distributed energy resources (DER) facilities and an associated [dashboard](https://maps.puco.ohio.gov/portal/apps/dashboards/ef2586cbf54b42cd8f5af3cf5c5da296). The dashboard provides the following notes:
  - A distributed energy resource (DER) is a source of electric power that is not directly connected to a bulk power system. DER includes both generators and energy storage technologies capable of exporting active power to the electric grid.
  - Energy Storage Capacity is reflective of standalone energy storage systems, not hybrid systems where capacity is already reported/captured in the generating units.
  - "Other" Fuel Types include Waste Gas, Biofuel, Diesel, Natural Gas/Propane, Coal, Cogeneration, and Hydro.
  
This notebook produces a tileset that includes a summary of DER facilities for the MORPC 15-county region and the counties and communities therein.  This notebook is the final stage in a pipeline that fetches, standardizes, and summarizes the DER facility data.

## Setup

### Load required libraries

In [ ]:
import pandas as pd
import frictionless
import os
import sys
import json
import datetime
import textwrap
import matplotlib
from matplotlib import pyplot as plt
sys.path.append(os.path.normpath("../morpc-common"))
import morpc

### User-specified parameters

In [ ]:
YEAR_RANGE = [2000, 2024]

### Static parameters

In [ ]:
INPUT_DIR = os.path.normpath("./input_data")

OUTPUT_DIR = os.path.normpath("./output_data")

CHART_DIRNAME = "charts"

### Define inputs

#### Create input data directory

Create input data directory if it doesn't exist.

In [ ]:
inputDir = os.path.normpath(INPUT_DIR)
if not os.path.exists(inputDir):
    os.makedirs(inputDir)

#### Summarized DER facilities data

In [ ]:
DER_INPUT_TABLE_RESOURCE = os.path.normpath("../morpc-renewenergyfacilities-summarize/output_data/morpc-renewenergyfacilities-der-long.resource.yaml")
print("Resource file: {}".format(DER_INPUT_TABLE_RESOURCE))

#### Geography lookup table [375]

In [ ]:
GEOS_LOOKUP_TABLE_RESOURCE = os.path.normpath("../morpc-geos-collect/output_data/morpc-geos-lookup.resource.yaml")
print("Resource file: {}".format(GEOS_LOOKUP_TABLE_RESOURCE))

#### MORPC member list [122]

In [ ]:
MEMBERS_DATA_PATH = "../morpc-lookup/Member_List.xlsx"
MEMBERS_SHEET = "Current Year Members"
MEMBERS_SCHEMA_PATH = "../morpc-lookup/Member_List_schema.json"
print("Data: {}, sheet '{}'".format(MEMBERS_DATA_PATH, MEMBERS_SHEET))
print("Schema: {}".format(MEMBERS_SCHEMA_PATH))

### Define outputs

#### Create output data directory

Create output data directory if it doesn't exist.

In [ ]:
outputDir = os.path.normpath(OUTPUT_DIR)
if not os.path.exists(outputDir):
    os.makedirs(outputDir)   

In [ ]:
chartDir = os.path.join(outputDir, CHART_DIRNAME)
if not os.path.exists(chartDir):
    os.makedirs(chartDir)    

#### DER facilities by year

In [ ]:
FACILITIES_TABLE_FILENAME = "renewenergy-der-long.csv"
FACILITIES_TABLE_PATH = os.path.join(outputDir, FACILITIES_TABLE_FILENAME)
FACILITIES_TABLE_SCHEMA_PATH = FACILITIES_TABLE_PATH.replace(".csv",".schema.yaml")
FACILITIES_TABLE_RESOURCE_PATH = FACILITIES_TABLE_PATH.replace(".csv",".resource.yaml")
print("Data: {}".format(FACILITIES_TABLE_PATH))
print("Schema: {}".format(FACILITIES_TABLE_SCHEMA_PATH))
print("Resource file: {}".format(FACILITIES_TABLE_RESOURCE_PATH))

## Prepare input data

### Load geography lookup table

In [ ]:
(geosRaw, geosRawResource, geosRawSchema) = morpc.frictionless_load_data(GEOS_LOOKUP_TABLE_RESOURCE, validate=True, archiveDir=inputDir)

In [ ]:
geosRaw.head()

In [ ]:
geos = geosRaw.copy()

### Load summarized DER facility data from upstream workflows

In [ ]:
(facilitiesRaw, facilitiesRawResource, facilitiesRawSchema) = morpc.frictionless_load_data(DER_INPUT_TABLE_RESOURCE, validate=True, archiveDir=inputDir)

In [ ]:
facilitiesRaw.head()

### MORPC member list

The output data will include all communities for whom data has not been suppressed, however we will only show data for MORPC members in the platform.

Load the member table.

In [ ]:
membersRaw = pd.read_excel(MEMBERS_DATA_PATH, sheet_name=MEMBERS_SHEET)
membersRaw.head()

Load the schema.

In [ ]:
membersSchema = morpc.load_avro_schema(MEMBERS_SCHEMA_PATH)

Verify that the fields are all the expected types.

In [ ]:
members = morpc.cast_field_types(membersRaw, membersSchema)

Extract only the communities which are themselves a member.

In [ ]:
members = members.loc[members["Local Member"] == True].copy()

The records in the member table are all county parts.  For places we need to subsitute the GEOID for the full place rather than the county part.

In [ ]:
members["PLACEFP"] = members["GEOID"].apply(lambda x:x[11:16])
members["COUSUBFP"] = members["GEOID"].apply(lambda x:x[14:19])
members["COUNTYID"] = members["County"].map(morpc.CONST_COUNTY_NAME_TO_ID)
members["GEOIDFQ"] = None
temp = members.loc[members["GovType"] == "Township"].copy()
temp["GEOIDFQ"] = temp["GEOID"]
members.update(temp)
temp = members.loc[members["GovType"] != "Township"].copy()
temp["GEOIDFQ"] = "1600000US39" + temp["PLACEFP"]
members.update(temp)

Now extract just the list of member GEOIDs.  The steps above likely produced duplicate records for places, so extract only the unique GEOIDs.

In [ ]:
memberList = list(members["GEOIDFQ"].unique())

We also need to append the list of counties.  We will include data for all counties regardless of membership status.

In [ ]:
memberList += ["0500000US{}".format(morpc.CONST_COUNTY_NAME_TO_ID[x]) for x in morpc.CONST_REGIONS['REGION15']]

Finally we need to append the ID for the MORPC region.

In [ ]:
memberList.append("M010000US001")

## Transform data to format required by Insights platform

In [ ]:
facilities = facilitiesRaw.loc[facilitiesRaw["YEAR"].isin(range(YEAR_RANGE[0], YEAR_RANGE[1]+1))].copy()
facilities["GEOTYPE"] = facilities["SUMLEVEL"].map(morpc.HIERARCHY_STRING_LOOKUP)

In [ ]:
facilities = facilities.drop(columns="NAME").merge(geos[["GEOIDFQ","COUNTYFP","NAME"]], on="GEOIDFQ")
facilities["COUNTYFP"] = "39" + facilities["COUNTYFP"]
facilities["COUNTY"] = facilities["COUNTYFP"].map(morpc.CONST_COUNTY_ID_TO_NAME)

In [ ]:
temp = facilities.loc[facilities["SUMLEVEL"] == "070"].copy()
temp["NAME"] = temp["NAME"] + " Township (" + temp["COUNTY"] + ")"
facilities.update(temp, overwrite=True, errors="ignore")

In [ ]:
temp = facilities.loc[facilities["SUMLEVEL"] == "050"].copy()
temp["NAME"] = temp["NAME"] + " County"
facilities.update(temp, overwrite=True, errors="ignore")

In [ ]:
facilities = facilities.filter(items=["GEOIDFQ","NAME","GEOTYPE","YEAR","METRIC","FUEL_TYPE","VALUE"], axis="columns")

In [ ]:
facilities = facilities.pivot(index=["GEOIDFQ","NAME","GEOTYPE","YEAR","FUEL_TYPE"], columns="METRIC", values="VALUE").reset_index()

In [ ]:
facilities.columns.name = None

In [ ]:
facilities = facilities.rename(columns={
        "Capacity":"CAPACITY",
        "Facilities":"FACILITIES"
    }) \
    .astype({
        "CAPACITY":"float",
        "FACILITIES":"int"
    })

In [ ]:
facilities = facilities.filter(items=["GEOIDFQ","NAME","GEOTYPE","YEAR","METRIC","FUEL_TYPE","FACILITIES","CAPACITY"], axis="columns")

In [ ]:
facilities.head()

## Export data

In [ ]:
facilities.to_csv(FACILITIES_TABLE_PATH, index=False)

## Create resource file for exported data

In [ ]:
facilitiesResource = morpc.frictionless_create_resource(FACILITIES_TABLE_FILENAME, 
    resourcePath=FACILITIES_TABLE_RESOURCE_PATH,
    title="MORPC Insights | Distributed Energy Resources Facilities by Year", 
    name="renewenergy_der", 
    description="Count and generation capacity of Central Ohio Distributed Energy Resources facilites which opened in each year according to data maintained by the Public Utilities Commission of Central Ohio.",
    writeResource=True,
    validate=True
)

## Generate static charts

In [ ]:
for f in os.scandir(chartDir):
    os.remove(f)

Load a standard color set for the chart elements.

In [ ]:
colorset = json.loads(json.dumps(morpc.CONST_COLOR_CYCLES["matplotlib"]))

### Facilities

In [ ]:
%matplotlib agg

# Create a list to accumulate geographies for which a thumbnail is generated
platformIncludeList = []
# Iterate over each geography in data set
for geoid in facilities["GEOIDFQ"].unique():
    # If the geography is not a MORPC member, skip it. The platform only features members.
    if(not geoid in memberList):
        continue
    
    # Extract the data for a single geography
    temp = facilities.loc[facilities["GEOIDFQ"] == geoid].copy()

    if(temp.empty):
        continue
        
    platformIncludeList.append(geoid)

    # Generate a title string
    geoName = temp.iloc[0]["NAME"]
    title = "Distributed Energy Resources Facilities by Year Opened - {}".format(geoName)
    xlabel = None
    ylabel = None
    
    # Drop the geography name and type
    temp = temp.filter(items=["YEAR","FUEL_TYPE","FACILITIES"], axis="columns")

    # Make the variable names nicer looking
    temp = temp.rename(columns={
        "YEAR":"Year",
        "FUEL_TYPE":"Fuel type",
        "FACILITIES":"Facilities"
    })
    labels = temp["Fuel type"].unique()
    labels = [textwrap.fill(label, 15) for label in labels]

    # Pivot to wide format
    temp = temp.pivot(index="Year", columns="Fuel type").reset_index()
    
    # Create and annotate the plot
    PLOTWIDTH = 8
    fig,ax = plt.subplots(figsize=(PLOTWIDTH,PLOTWIDTH/16*9))

    temp.plot.bar(ax=ax, x="Year", y="Facilities", stacked=True, color=colorset)
    temp = facilities.loc[facilities["GEOIDFQ"] == geoid].copy()
    
    ax.set_title(title, fontsize=14)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_yticks([round(tick,0) for tick in ax.get_yticks()])
    legend = ax.legend(labels, loc='center left', bbox_to_anchor=(1, 0.5), labelspacing=1)
    ax.grid(visible=True, color="lightgrey")
    ax.set_axisbelow(True)
    
    # Format the y-axis labels as integers with comma separators
    ax.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
    
    # Save the figure to disk as an SVG file
    ax.figure.savefig(os.path.join(chartDir, "facilities-{}.svg".format(geoid)), bbox_extra_artists=(legend,), bbox_inches='tight')
    
    plt.close(ax.figure)

    excelData = temp[["YEAR","FUEL_TYPE","CAPACITY"]].pivot(index="YEAR", columns="FUEL_TYPE", values="CAPACITY")
    excelData = excelData.rename(columns={
        "FACILITIES":"Facilities"
    })
    excelData.index.name = "Open year"
    excelData.columns.name = None

    writer = pd.ExcelWriter(os.path.join(chartDir, "facilities-{}.xlsx".format(geoid)), engine='xlsxwriter')
    dataOptions = {
        "numberFormat": {
            'Open year': "0",
            'Biofuel': "#,##0",
            'Energy Storage': "#,##0",
            'Natural Gas/Propane': "#,##0",
            'Solar': "#,##0",
            'Waste Gas': "#,##0",
            'Wind': "#,##0"
        },
        "columnWidth": 20
    }
    chartOptions = {
        "subtype":"stacked",
        "colors": colorset,
        "titles": {
            "chartTitle": title,
            "xTitle": xlabel,
            "yTitle": ylabel
        },
        "seriesOptions": [{"gap":100} for x in excelData.columns],
        "xAxisOptions": {
            "num_font": {"size":14},
        },
        "yAxisOptions": {
            "num_font": {"size":14},
            "num_format": "#,##0",
        },
        "legendOptions":{
            "position":"bottom",
            "font":{"size":14}
        },
        "sizeOptions":{
            "x_scale":1.5,
            "y_scale":1.5
        }
    }
    morpc.data_chart_to_excel(excelData, writer, chartType="column", dataOptions=dataOptions, chartOptions=chartOptions)
    writer.close()    
    
%matplotlib inline

### Capacity

In [ ]:
%matplotlib agg

# Create a list to accumulate geographies for which a thumbnail is generated
platformIncludeList = []

# Iterate over each geography in data set
for geoid in facilities["GEOIDFQ"].unique():
    # If the geography is not a MORPC member, skip it. The platform only features members.
    if(not geoid in memberList):
        continue
    
    # Extract the data for a single geography
    temp = facilities.loc[facilities["GEOIDFQ"] == geoid].copy()

    if(temp.empty):
        continue
        
    platformIncludeList.append(geoid)

    # Generate a title string
    geoName = temp.iloc[0]["NAME"]
    title = "Distributed Energy Resources Capacity by Year Opened - {}".format(geoName)
    xlabel = None
    ylabel = "Kilowatts (kW)"
    
    # Drop the geography name and type
    temp = temp.filter(items=["YEAR","FUEL_TYPE","CAPACITY"], axis="columns")
    
    # Make the variable names nicer looking
    temp = temp.rename(columns={
        "YEAR":"Year",
        "FUEL_TYPE":"Fuel type",
        "CAPACITY":"Capacity"
    })
    labels = temp["Fuel type"].unique()
    labels = [textwrap.fill(label, 15) for label in labels]

    # Pivot to wide format
    temp = temp.pivot(index="Year", columns="Fuel type").reset_index()
    
    # Create and annotate the plot
    PLOTWIDTH = 8
    fig,ax = plt.subplots(figsize=(PLOTWIDTH,PLOTWIDTH/16*9))

    temp.plot.bar(ax=ax, x="Year", y="Capacity", stacked=True, color=colorset)
    temp = facilities.loc[facilities["GEOIDFQ"] == geoid].copy()
    
    ax.set_title(title, fontsize=14)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_yticks([round(tick,0) for tick in ax.get_yticks()])
    legend = ax.legend(labels, loc='center left', bbox_to_anchor=(1, 0.5), labelspacing=1)
    ax.grid(visible=True, color="lightgrey")
    ax.set_axisbelow(True)
    
    # Format the y-axis labels as integers with comma separators
    ax.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
    
    # Save the figure to disk as an SVG file
    ax.figure.savefig(os.path.join(chartDir, "capacity-{}.svg".format(geoid)), bbox_extra_artists=(legend,), bbox_inches='tight')
    
    plt.close(ax.figure)

    excelData = temp[["YEAR","FUEL_TYPE","CAPACITY"]].pivot(index="YEAR", columns="FUEL_TYPE", values="CAPACITY")
    excelData = excelData.rename(columns={
        "CAPACITY":"Capacity"
    })
    excelData.index.name = "Open year"
    excelData.columns.name = None

    writer = pd.ExcelWriter(os.path.join(chartDir, "capacity-{}.xlsx".format(geoid)), engine='xlsxwriter')
    dataOptions = {
        "numberFormat": {
            'Open year': "0",
            'Biofuel': "#,##0.0",
            'Energy Storage': "#,##0.0",
            'Natural Gas/Propane': "#,##0.0",
            'Solar': "#,##0.0",
            'Waste Gas': "#,##0.0",
            'Wind': "#,##0.0"
        },
        "columnWidth": 20
    }
    chartOptions = {
        "subtype":"stacked",
        "colors": colorset,
        "titles": {
            "chartTitle": title,
            "xTitle": xlabel,
            "yTitle": ylabel
        },
        "seriesOptions": [{"gap":100} for x in excelData.columns],
        "xAxisOptions": {
            "num_font": {"size":14},
        },
        "yAxisOptions": {
            "num_font": {"size":14},
            "num_format": "#,##0",
        },
        "legendOptions":{
            "position":"bottom",
            "font":{"size":14}
        },
        "sizeOptions":{
            "x_scale":1.5,
            "y_scale":1.5
        }
    }
    morpc.data_chart_to_excel(excelData, writer, chartType="column", dataOptions=dataOptions, chartOptions=chartOptions)
    writer.close()    
    
%matplotlib inline

## Generate Insights catalog content

The content in the Insights platform is controlled by a catalog spreadsheet. Each tile to be displayed in the platform must have a record in the catalog.  This section will create the records for the tiles that display the alternative fuel station data.  Eventually this function will be performed by a separate staging script.

First specify the column names used in the catalog.

In [ ]:
columnNames=["TileID","TilesetID","GeographyType","GeographyName","Category","Headline","Commentary","ThumbnailURL","Contributor","Vintage","UpdateInterval","ShareURL","DataProductURL","MoreInformationURL"]

Create a new dataframe containing only the geographies for which thumbnail images were produced in the section above.

In [ ]:
catalog = facilities.loc[facilities["GEOIDFQ"].isin(platformIncludeList)].copy()

Extract only the metadata columns of interest and flatten the data to have only one record per geography. Rename the metadata fields to match the catalog fields.

In [ ]:
catalog = catalog.filter(items=["GEOIDFQ","NAME","GEOTYPE"], axis="columns") \
    .groupby("GEOIDFQ").first() \
    .reset_index() \
    .rename(columns={"NAME":"GeographyName","GEOTYPE":"GeographyType"})

Change the GeographyType values to match the schema of the catalog.

In [ ]:
catalog["GeographyType"] = catalog["GeographyType"].map({
    "REGION15":"Region",
    "COUNTY":"County",
    "COUNTY-TOWNSHIP-REMAINDER":"Community",
    "PLACE":"Community"
})

Populate some placeholder fields.

In [ ]:
catalog["TileID"] = None
catalog["TilesetID"] = "TBD-Facilities"
catalog["Category"] = None
catalog["Headline"] = "TBD"
catalog["Commentary"] = "TBD"

Generate the URL for the thumbnail images. These will be hosted in GitHub and will be indexed by GEOIDFQ.

In [ ]:
catalog["ThumbnailURL"] = catalog["GEOIDFQ"].apply(lambda geoid:"https://raw.githubusercontent.com/morpc-insights/renewenergy-der/refs/heads/main/output_data/charts/facilities-{}.svg".format(geoid))

Populate some other simple metadata.  Vintage in this case refers to the year that the content was published in Insights. This is to give readers an idea of how old it is.  UpdateInterval gives them an idea of when to expect the next version. ShareURL is a placeholder for now.

In [ ]:
catalog["Contributor"] = "Mid-Ohio Regional Planning Commission"
catalog["Vintage"] = str(datetime.date.today().year)
catalog["UpdateInterval"] = "annually"
catalog["ShareURL"] = None

Generate the data product URL.  This points to an ArcGIS Dashboard that accepts URL parameters.  GEOIDFQ is passed as a parameter to tell the app to load the data for a particular geography.

In [ ]:
catalog["DataProductURL"] = catalog["GEOIDFQ"].apply(lambda geoid:"https://www.arcgis.com/apps/dashboards/3f2b48c930294cfda824567333f001fd#geoid={}".format(geoid))

Generate the URLs that point to the extended commentary pages.  Default to a common page (population.pdf) hosted in GitHub.  Point to specific pages for the 15-county region and for each county.

In [ ]:
catalog["MoreInformationURL"] = "https://raw.githubusercontent.com/morpc-insights/renewenergy-der/refs/heads/main/fact_sheets/renewenergy-der.pdf"

Extract only the required columns.

In [ ]:
catalog = catalog.filter(items=columnNames, axis="columns")

Inspect the listing.

In [ ]:
catalog.head()

In [ ]:
temp = catalog.copy()

In [ ]:
temp["ThumbnailURL"] = temp["ThumbnailURL"].str.replace("facilities-", "capacity-")

In [ ]:
temp["TilesetID"] = temp["TilesetID"].str.replace("Facilities", "Capacity")

In [ ]:
catalog = pd.concat([catalog, temp], axis="index")

Save the catalog to an Excel spreadsheet.

In [ ]:
catalog.to_excel("catalog.xlsx", index=False)

It is necessary to manually add these records to the master catalog or update the records already therein.  See the following file in GitHub:

https://github.com/morpc/morpc-insights/blob/main/catalog/morpc_insights_catalog.xlsx
